In [14]:
# Sorry this part is messy Maxim, it's test data!
# Each tuple composes of 2 internal tuples: (start_point, end_point)
# Start_point and end_point are defined as (j, i). This ordering is how cityflow 
# defines its intersections, so I kept it the same for consistency

# I sent a diagram in discord so the test data I'm using makes sense. Red circles are intersections

edges_east_south = [((0,2), (1,2)), ((1,2), (2,2)), ((2,2), (3,2)), ((0,1), (1,1)), ((1,1), (2,1)), ((1,2), (1,1)), ((2,2), (2,1)), ((1,1), (1,0))]
edges_west_north = [((1,2), (0,2)), ((2,2), (1,2)), ((3,2), (2,2)), ((1,1), (0,1)), ((2,1), (1,1)), ((1,1), (1,2)), ((2,1), (2,2)), ((1,0), (1,1))]

edges = edges_east_south + edges_west_north

print(edges)

[((0, 2), (1, 2)), ((1, 2), (2, 2)), ((2, 2), (3, 2)), ((0, 1), (1, 1)), ((1, 1), (2, 1)), ((1, 2), (1, 1)), ((2, 2), (2, 1)), ((1, 1), (1, 0)), ((1, 2), (0, 2)), ((2, 2), (1, 2)), ((3, 2), (2, 2)), ((1, 1), (0, 1)), ((2, 1), (1, 1)), ((1, 1), (1, 2)), ((2, 1), (2, 2)), ((1, 0), (1, 1))]


In [15]:
# Problem: Cityflow loops through i, j, and k to create a grid. 
# Need to convert custom graph edges into roadnet structure.

# Assume 2-way traffic along each edge
# Current loop requires a full list of edges (e.g. both 1->2 AND 2->1)

def gridToRoadnet(edges, rowDistances, columnDistances, outRowDistance, outColumnDistance,
                  intersectionWidths, laneWidth=4, laneMaxSpeed=20,
                  numLeftLanes=1, numStraightLanes=1, numRightLanes=1, tlPlan=False, midPoints=10):

    
    #### Unchanged from cityflow#############################
    numLanes = numLeftLanes + numStraightLanes + numRightLanes
    
    x = [[None for _ in range(colNum)] for _ in range(rowNum)]
    y = [[None for _ in range(colNum)] for _ in range(rowNum)]

    rowDistances = [outRowDistance] + rowDistances + [outRowDistance]
    columnDistances = [outColumnDistance] + columnDistances + [outColumnDistance]
    
    def pointToDict(x, y):
        return {"x": x, "y": y}

    for i in range(rowNum):
        for j in range(colNum):
            if j > 0:
                x[i][j] = x[i][j - 1] + rowDistances[i - 1]
                y[i][j] = y[i][j - 1]
            elif i > 0:
                x[i][j] = x[i - 1][j]
                y[i][j] = y[i - 1][j] + columnDistances[j - 1]
            else:
                x[i][j] = -outRowDistance
                y[i][j] = -outColumnDistance
                
    
     ###########################################################           
    
    #Create as many roads/intersections as edges (as opposed to as many roads in a ixj grid)
    roads = [[None, None, None, None] for _ in range(len(edges))]
    intersections = [None for _ in range(len(edges))]

    
    for index, pair in enumerate(edges):

        i = pair[0][1]
        j = pair[0][0]
        ni = pair[1][1]
        nj = pair[1][0]

        if ni == i and nj > j:
        #traveling east
            k = 0
        elif ni == i and nj < j:
        #traveling west 
            k = 2
        elif nj == j and ni > i:
        #traveling north
            k = 1
        else:
        #traveling south
            k = 3

        road = {
            "id": "road_%d_%d_%d" % (j, i, k),
            "direction": k,
            "fromi": i,
            "fromj": j,
            "toi": ni,
            "toj": nj,
            "points": [
                pointToDict(x[i][j], y[i][j]),
                pointToDict(x[ni][nj], y[ni][nj])
            ],
            "lanes": [
                {
                    "width": laneWidth,
                    "maxSpeed": laneMaxSpeed
                }
            ] * numLanes,
            "startIntersection": "intersection_%d_%d" % (j, i),
            "endIntersection": "intersection_%d_%d" % (nj, ni)
        }
        roads[index] = road


In [16]:
#These metrics pulled from ```generate_roadnet``` for a 3 x 4 grid


rowNum = 3
colNum = 4
rowDistance = 300
columnDistance = 300
intersectionWidth = 100
rowDistances = [rowDistance] * (colNum - 1)
columnDistances = [columnDistance] * (rowNum - 1)
outRowDistance = rowDistance
outColumnDistance = columnDistance
intersectionWidths = [[intersectionWidth] * colNum] * rowNum


gridToRoadnet(edges, rowDistances, columnDistances, outRowDistance, outColumnDistance,
                  intersectionWidths, laneWidth=4, laneMaxSpeed=20,
                  numLeftLanes=1, numStraightLanes=1, numRightLanes=1, tlPlan=False, midPoints=10)